# Model Test

⚠ Work In Progress ⚠

In [2]:
import os
import numpy as np
import json
import pandas as pd
from dataset_analyzer import BirdAnalyzer

# Validation Results
Introduction of all the variables to start the validation process.

This involves:

1. Evaluation and extraction of the predicted segments in validation set
2. Threshold computation from validation results

In [21]:
# check 1_segments_extraction for true_segments_test
DATASET_NAME = 'NEW_DATASET_1'
DATASET_PATH = f'E:/Giacomo/Tovanella/{DATASET_NAME}'
MODEL_NAME = "pasqualo"

with open(f'utils/{DATASET_NAME}/true_segments.json') as f:
    true_segments = json.load(f)

with open(f'utils/WABAD/true_segments_wabad.json') as f:
    true_segments_wabad = json.load(f)

In [22]:
# !!! remember to remove non-species class in CustomClassifierLabels
if not os.path.exists(f"classifiers/official/{MODEL_NAME}/CustomClassifier_Labels.txt_old"):
    labels_path = f"classifiers/official/{MODEL_NAME}/CustomClassifier_Labels.txt"
    labels_path_1 = f"classifiers/official/{MODEL_NAME}/CustomClassifier_Labels_1.txt"
    with open(labels_path, 'r') as infile, open(labels_path_1, 'w') as outfile:
        for line in infile:
            line = line.strip()
            if "_" in line:
                outfile.write(line + '\n')
            else:
                outfile.write(line + '_\n')

    os.rename(labels_path, labels_path + "_old")
    os.rename(labels_path_1, labels_path)

## Evaluation and Extraction from Validation

In [23]:
bird_analyzer = BirdAnalyzer(MODEL_NAME, DATASET_PATH, 0.1)
valid_done = False
test_done = False

load_custom_models
Custom model loaded.
loading custom classifier labels
Labels loaded.
load model False
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [24]:
if not valid_done:
    valid_pred_segments = bird_analyzer.process_data_set("valid")
valid_done = True

Starting Watcher for valid set, folder: Certhia familiaris_Eurasian Treecreeper
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_490_5.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_490_5.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_495_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_495_0.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_498_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_498_0.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190603_030000_534_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\valid\Certhia familiaris_Eurasian Treecreeper\20190603_030000_534_0.wav
read_audio_data
read_audio_data: com

In [25]:
# LOAD
with open(f"classifiers/official/{MODEL_NAME}/valid_pred_segments.json", "w") as f:
    json.dump(valid_pred_segments, f)

In [26]:
# SAVE
with open(f"classifiers/official/{MODEL_NAME}/valid_pred_segments.json") as f:
    valid_pred_segments = json.load(f)

## Threshold Computation from Validation results

Da ricordare che in Validation sono presenti WABAD e originali. Per calcolare le soglie di confidenza, devo capire se la predizione del modello è corretta e quale soglia associargli.

In conf_scores.json vado a inserire tutti i confidence scores relativi ad una specie e gli assegno True se il modello ha predetto correttamente, False altrimenti.

In [27]:
valid_species_list = os.listdir(f"{DATASET_PATH}/valid")

In [28]:
conf_scores = { species: [] for species in valid_species_list }
for audio in valid_pred_segments.keys():                            # {'20190603_030000.WAV': {'541_5': {'species_B': 0.256, '<species_A>': 0.195,
    for segm, species in valid_pred_segments[audio].items():        # {'541_5': {'species_B': 0.256, '<species_A>': 0.195,
        for sp in species:
            if sp not in valid_species_list:
                continue
            score = valid_pred_segments[audio][segm][sp]
            if score <= 0.15:
                continue
            is_wabad = len(audio.split("_")) == 3                   # <BAM>_<20151130>_<091318.WAV>
            if is_wabad:
                is_correct = sp in true_segments_wabad[audio][segm]
            else:
                is_correct = sp in true_segments[audio][segm]
            conf_scores[sp].append([score, is_correct])

In [29]:
with open(f"classifiers/official/{MODEL_NAME}/conf_scores.json", "w") as f:
    json.dump(conf_scores, f)

In [30]:
from sklearn.metrics import precision_recall_fscore_support

best_threshold_by_species = {}
default_threshold = 0.15
for specie, data in conf_scores.items():
    if len(data) <= 10:
        best_threshold_by_species[specie] = default_threshold
        continue
    scores = np.array([x[0] for x in data])
    labels = np.array([x[1] for x in data])

    best_threshold = 0
    best_f1 = 0
    thresholds = np.linspace(scores.min(), scores.max(), 50)  # 100 soglie testate

    for threshold in thresholds:
        preds = scores >= threshold
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    best_threshold_by_species[specie] = best_threshold
    print(f"Optimal threshold for {specie}: {best_threshold:.3f} with f1 score: {best_f1:.3f}")

Optimal threshold for Certhia familiaris_Eurasian Treecreeper: 0.195 with f1 score: 0.622
Optimal threshold for Erithacus rubecula_European Robin: 0.167 with f1 score: 0.827
Optimal threshold for Fringilla coelebs_Common Chaffinch: 0.150 with f1 score: 0.910
Optimal threshold for Lophophanes cristatus_Crested Tit: 0.167 with f1 score: 0.789
Optimal threshold for Loxia curvirostra_Common Crossbill: 0.190 with f1 score: 1.000
Optimal threshold for Muscicapa striata_Spotted Flycatcher: 0.179 with f1 score: 0.984
Optimal threshold for Periparus ater_Coal Tit: 0.170 with f1 score: 0.846
Optimal threshold for Phylloscopus collybita_Common Chiffchaff: 0.151 with f1 score: 0.919
Optimal threshold for Regulus ignicapilla_Common Firecrest: 0.150 with f1 score: 0.895
Optimal threshold for Regulus regulus_Goldcrest: 0.151 with f1 score: 0.878
Optimal threshold for Sylvia atricapilla_Eurasian Blackcap: 0.150 with f1 score: 0.938
Optimal threshold for Troglodytes troglodytes_Eurasian Wren: 0.150 wit

In [31]:
# save best_threshold_by_species
with open(f"classifiers/official/{MODEL_NAME}/best_threshold_by_species.json", "w") as f:
    json.dump(best_threshold_by_species, f)

# Test Results

In [ ]:
if not test_done:
    test_pred_segments = bird_analyzer.process_data_set("test")
test_done = True

Starting Watcher for test set, folder: Certhia familiaris_Eurasian Treecreeper
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_189_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_189_0.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_190_5.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_190_5.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_192_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_192_0.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190608_070000_246_0.wav
Analyzing  E:\Giacomo\Tovanella\NEW_DATASET_1\test\Certhia familiaris_Eurasian Treecreeper\20190608_070000_246_0.wav
read_audio_data
read_audio_data: complete

# Test Evaluation

In [ ]:
def filter_predictions_by_threshold(pred_segments, best_threshold_by_species):
    filtered_pred_segments = {}
    for audio, segments in pred_segments.items():
        filtered_pred_segments[audio] = {}
        for segm_id, predictions in segments.items():
            species_to_save = [
                label for label, conf_score in predictions.items()
                if conf_score > best_threshold_by_species.get(label, 0)
            ]
            filtered_pred_segments[audio][segm_id] = species_to_save
    return filtered_pred_segments


In [ ]:
filtered_pred_segments = filter_predictions_by_threshold(test_pred_segments, best_threshold_by_species)

In [ ]:
test_species_list = os.listdir(f"{DATASET_PATH}/test")

In [ ]:
# extract only test segments from true_segments
test_true_segments = {}
for audio in filtered_pred_segments.keys():                         # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],
    test_true_segments[audio] = {}                                  # {'20190608_070000.WAV' : {}}
    for key, values in filtered_pred_segments[audio].items():       #  {'189_0': ['Poecile palustris_Marsh Tit'],
        test_true_segments[audio][key] = [value for value in true_segments[audio][key] if value in test_species_list]  # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([list(test_species_list)])

len(mlb.classes_)

17

In [ ]:
y_pred = []
y_true = []
for audio in test_pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in test_true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in test_pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Spinus spinus_Eurasian Siskin', 'Turdus viscivorus_Mistle Thrush'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Pecking_', 'Vegetation_'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Rain_'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['None_', 'Pecking_'] will be ignored
  warnings.warn(
c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Turdus viscivorus_Mistle Thrush'

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)
with open(f"classifiers/official/{MODEL_NAME}/classification_report.json", "w") as f:
    json.dump(report, f)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Certhia familiaris_Eurasian Treecreeper,1.000000,0.078947,0.146341,76.0
Dendrocopos major_Great Spotted Woodpecker,1.000000,0.296296,0.457143,27.0
Dryocopus martius_Black Woodpecker,1.000000,0.083333,0.153846,24.0
Erithacus rubecula_European Robin,0.383178,0.634021,0.477670,194.0
Fringilla coelebs_Common Chaffinch,0.659890,0.444486,0.531181,2702.0
Glaucidium passerinum_Eurasian Pygmy-Owl,0.500000,0.592593,0.542373,27.0
Lophophanes cristatus_Crested Tit,0.650000,0.302326,0.412698,43.0
Loxia curvirostra_Common Crossbill,0.307018,0.583333,0.402299,60.0
Muscicapa striata_Spotted Flycatcher,0.622378,0.559748,0.589404,159.0
Periparus ater_Coal Tit,0.005967,0.033113,0.010111,151.0
